# TripAdvisor scraping com Python e Selenium

## Importando as bibliotecas necessárias

In [1]:
from selenium import webdriver
from hotelsInfo import HotelsInfo
import re
import pandas as pd

## Abrindo o arquivo de links dos hotéis e o browser

In [3]:
df = pd.read_excel('Links Hoteis MT.xlsx')
driver = webdriver.Firefox()

## Definindo as funcões de coleta de dados

In [4]:
def getAllInformations():
    city,date = getCityAndDate()
    listHotels = getListOfAvailableHotels()
    getHotelsInfo(city, date, listHotels)

In [5]:
def getCityAndDate():
    city = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div[6]/div[2]/div/div[2]/div[1]/div/div/h1').text
    date = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div[6]/div[2]/div/div[2]/div[2]/div/div/div/div[1]/div/div/div[1]/button[1]/div/span[2]/span[2]').text
    return city, date

In [6]:
def getListOfAvailableHotels():
    availableHotels = driver.find_element_by_id('taplc_hsx_hotel_list_lite_dusty_hotels_combined_sponsored_0')
    listHotels = availableHotels.find_elements_by_css_selector('div.ui_column.is-8.main_col.allowEllipsis')
    return listHotels

In [7]:
try:
    del h
    h = HotelsInfo()
except:
    h = HotelsInfo()
numHotel = 0

In [8]:
def getHotelsInfo(city, date, listHotels):
    global numHotel
    print(city+" - "+str(len(listHotels)))
    numHotel += len(listHotels)
    if len(listHotels) > 0:
        for hotel in listHotels:
            try:
                hotelName = hotel.find_element_by_css_selector('a.property_title.prominent').text
                hotelDailyRate = re.findall('\d[,.]\d+|\d+', hotel.find_element_by_css_selector('div.price.__resizeWatch').text)
                hotelDailyRate = '-' if len(hotelDailyRate) == 0 else hotelDailyRate[0]
                hotelReviewCount = re.findall('\d[,.]\d+|\d+', hotel.find_element_by_css_selector('a.review_count').text)
                hotelReviewCount = '-' if len(hotelReviewCount) == 0 else hotelReviewCount[0]
                hotelDailyDate = re.search('\d+/\d+/\d+', date).group(0)
                hotelCity = city.replace("Hotéis em ", "")
                h.add(hotelName, hotelDailyRate, hotelCity, hotelReviewCount, hotelDailyDate)
            except:
                pass
            #h.finish()
    #print(h.hotels.head())

## Abrindo o site no primeiro link

In [9]:
driver.get(df.linkHoteis[0])

## Coletando e salvando os dados dos hotéis de todos os links

### *Selecionar os parâmetros de check-in, check-out e número de hóspedes antes de prosseguir

In [10]:
for link in df.linkHoteis[:]:
    driver.implicitly_wait(3)
    driver.get(link)
    driver.implicitly_wait(4)
    getAllInformations()
print("Total de hotéis: "+str(numHotel))
_,date = getCityAndDate()
date = re.search('\d+/\d+/\d+', date).group(0)
date = date.replace("/", '-')
h.finish(date)
driver.quit()

Hotéis em Cuiabá - 31
Hotéis em Diamantino - 0
Total de hotéis: 31
